In [1]:
import numpy as np
import pandas as pd
month_num = 5
taxi_trip=pd.read_csv(r'E:\data ming\accident_prediction\NYC_data\taxi_data\yellow_tripdata_2017-'+'%02d'%month_num+'.csv')


In [2]:
Month_day = [31,28,31,30,31,30,31,31,30,31,30,31]
taxi_trip = taxi_trip.drop(columns=['store_and_fwd_flag','tolls_amount','payment_type','improvement_surcharge','tip_amount','extra','fare_amount'])
path="E:/data ming/accident_prediction/NYC_data/taxi_data/taxi_loca11.csv"
all_data=pd.DataFrame(pd.read_csv(path))
index_num=np.zeros([266,2]) #建立空矩阵 存放索引号-索引内容
index_num[all_data['LocationID'],0]=all_data['LocationID'] #填充索引内容
index_num[all_data['LocationID'],1]=all_data['grid_id'] #填充索引内容

In [3]:
len(taxi_trip)

10102124

In [4]:
#path="E:/data ming/accident_prediction/NYC_data/taxi_data/yellow_tripdata_2017-01.csv"
taxi_data=taxi_trip
#索引成功后的数组
PULocationGrid_ID=[]
DOLocationGrid_ID=[]
Month1 = []
Day1 = []
Min1 = []
Min2 = []
Pick_time = []
Drop_time = []


    
    
for i in range(0,len(taxi_data)):
    PULocationGrid_ID.append(index_num[taxi_data['PULocationID'][i],1])
    DOLocationGrid_ID.append(index_num[taxi_data['DOLocationID'][i],1])
    
    datatime0 = pd.to_datetime(taxi_data['tpep_pickup_datetime'][i])
    datatime1 = pd.to_datetime(taxi_data['tpep_dropoff_datetime'][i])
    
    pick_time = datatime0.hour*60+datatime0.minute
    Pick_time.append(pick_time)

    drop_time = datatime1.hour*60+datatime1.minute
    Drop_time.append(drop_time)
   
    Month1.append(datatime0.month)
    Day1.append(datatime0.day)
    
    Min1.append(pick_time/1440)
    Min2.append(drop_time/1440)
    
#保存为原矩阵中的内容，增加两列。
taxi_data['pick_ID']=PULocationGrid_ID  
taxi_data['drop_ID']=DOLocationGrid_ID 
taxi_data['Month']=Month1
taxi_data['Day']=Day1
taxi_data['min1']=Min1
taxi_data['min2']=Min2
taxi_data['pick_time']=Pick_time
taxi_data['drop_time']=Drop_time
#将结果输出


In [5]:
taxi_data = taxi_data.sort_values(['Day','pick_time'],ascending= [True,True])
pd.DataFrame(taxi_data).to_csv('E:/data ming/accident_prediction/NYC_data/taxi_17'+'%02d'%month_num+'.csv')#直接输出网格index
taxi_data=pd.DataFrame(pd.read_csv('E:/data ming/accident_prediction/NYC_data/taxi_17'+'%02d'%month_num+'.csv'))
day_num= Month_day[month_num-1]
Interval=10 #10分钟
num_interval=np.int(1440/Interval) 

In [19]:
np.unique(taxi_data['Day'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], dtype=int64)

In [6]:
#对一个月的数据做处理 每天30min一个interval 一天48个 31*48
import collections
taxi_halfhour_pick= np.zeros([day_num,num_interval,27,27])#生成数组保存每组速度
taxi_halfhour_drop= np.zeros([day_num,num_interval,27,27])#
ss=np.int32(0)
for ii in range(1,day_num+1):
    taxi_dataset=taxi_data[taxi_data['Day']==np.float(ii)]
    count_half_hour=np.zeros(num_interval+1)  #生成数组保存每组的个数，每半小时一个为48 下面为30分钟 27*27是网格大小
    for j in range(0,num_interval):# 分段，计算每个interval中有几条记录 然后每次索引时用这个数据来控制归类的数据条目段
        ab=np.where((taxi_dataset['pick_time']>=Interval*j)&(taxi_dataset['pick_time']<Interval*j+Interval),1,0)
        count_half_hour[j+1]=collections.Counter(ab)[1]
    for j in range(0,num_interval):
        #speed_value_ingrid=np.zeros(729)
        count_pick_id=np.zeros(729)
        count_drop_id=np.zeros(729)
        for i in range(ss,ss+np.int32(count_half_hour[j+1])):
            #print(ii,i,j)
            count_pick_id[np.int(taxi_dataset['pick_ID'][i])]=count_pick_id[np.int(taxi_dataset['pick_ID'][i])]+1
            count_drop_id[np.int(taxi_dataset['drop_ID'][i])]=count_pick_id[np.int(taxi_dataset['drop_ID'][i])]+1
        ss=np.int32(ss)+np.int32(count_half_hour[j+1])#ss为总的累计值
        #print(ss)
       
        taxi_halfhour_pick[ii-1,j,:,:]=count_pick_id.reshape(27,27)
        taxi_halfhour_drop[ii-1,j,:,:]=count_drop_id.reshape(27,27)
    print(ss)

#映射到三维当中去
taxi_halfhour_pick1= np.zeros([day_num*num_interval,27,27])
taxi_halfhour_drop1= np.zeros([day_num*num_interval,27,27])
for i in range(0,day_num):
    for j in range(0,num_interval):
        taxi_halfhour_pick1[i*num_interval+j,:,:]=taxi_halfhour_pick[i,j,:,:]
        taxi_halfhour_drop1[i*num_interval+j,:,:]=taxi_halfhour_drop[i,j,:,:]

303360
635926
981948
1337525
1700307
2070806
2388833
2695812
3026291
3366660
3714906
4068998
4440148
4737221
5049665
5386883
5738959
6088147
6452433
6820098
7139505
7446814
7770448
8107026
8449471
8755528
9022933
9264343
9477697
9774335
10102124


In [8]:
taxi_halfhour_pick1= np.zeros([day_num*num_interval,27,27])
taxi_halfhour_drop1= np.zeros([day_num*num_interval,27,27])
for i in range(0,day_num):
    for j in range(0,num_interval):
        taxi_halfhour_pick1[i*num_interval+j,:,:]=taxi_halfhour_pick[i,j,:,:]
        taxi_halfhour_drop1[i*num_interval+j,:,:]=taxi_halfhour_drop[i,j,:,:]
for i in range(day_num*num_interval):
    taxi_halfhour_pick1[i,0,0]=0
    taxi_halfhour_drop1[i,0,0]=0
index_road=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/Experimental_data/index_road.csv"))
index_road=np.array(index_road)[:,0]
def transfer_354(A):
    B1=np.zeros([354])
    A=np.array(A).reshape([729])
    for i in range(354):
        B1[i]=A[index_road[i]]
    return B1
taxi_halfhour_pick1_04=np.zeros([354,day_num*num_interval])
taxi_halfhour_drop1_04=np.zeros([354,day_num*num_interval])
for i in range(day_num*num_interval):
    taxi_halfhour_pick1_04[:,i]=transfer_354(taxi_halfhour_pick1[i,:,:])
    taxi_halfhour_drop1_04[:,i]=transfer_354(taxi_halfhour_drop1[i,:,:])
np.save('taxi_halfhour_pick_'+'%02d'%month_num+'.npy',taxi_halfhour_pick1_04)
np.save('taxi_halfhour_drop_'+'%02d'%month_num+'.npy',taxi_halfhour_drop1_04)

In [34]:
Taxi_trip_04=np.load(r'E:\data ming\accident_prediction\NYC_data\Experimental_data\taxi_pick_drop\taxi_pick_2017_04.npy')

In [31]:
taxi_halfhour_pick1[0,10,10]

29.0

In [37]:
Taxi_trip_04.shape

(354, 720)

In [103]:
month_num

3

In [53]:
sum(taxi_halfhour_pick1_04[:,0])+sum(taxi_halfhour_pick1_04[:,1])

17170.0

In [65]:
sum(Taxi_trip_04[:,-1])

8666.0

In [120]:
import numpy as np
import pandas as pd
taxi_trip_05=pd.read_csv(r'E:\data ming\accident_prediction\NYC_data\taxi_data\yellow_tripdata_2017-05.csv')
path="E:/data ming/accident_prediction/NYC_data/taxi_data/taxi_loca11.csv"
all_data=pd.DataFrame(pd.read_csv(path))
index_num=np.zeros([266,2]) #建立空矩阵 存放索引号-索引内容
index_num[all_data['LocationID'],0]=all_data['LocationID'] #填充索引内容
index_num[all_data['LocationID'],1]=all_data['grid_id'] #填充索引内容

#path="E:/data ming/accident_prediction/NYC_data/taxi_data/yellow_tripdata_2017-01.csv"
taxi_data=taxi_trip_05
#索引成功后的数组
PULocationGrid_ID=[]
DOLocationGrid_ID=[]
for i in range(0,len(taxi_data)):
    PULocationGrid_ID.append(index_num[taxi_data['PULocationID'][i],1])
    DOLocationGrid_ID.append(index_num[taxi_data['DOLocationID'][i],1])
#保存为原矩阵中的内容，增加两列。
taxi_data['PULocationGrid_ID']=PULocationGrid_ID  
taxi_data['DOLocationGrid_ID']=DOLocationGrid_ID  
#将结果输出
pd.DataFrame(taxi_data).to_csv('E:/data ming/accident_prediction/NYC_data/taxi_1705.csv')#直接输出网格index

In [121]:
taxi_trip_05=pd.read_csv(r'E:\data ming\accident_prediction\NYC_data\taxi_1705.csv')

In [122]:
taxi_trip_05

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationGrid_ID,DOLocationGrid_ID
0,0,2,2017-05-18 00:36:42,2017-05-18 00:38:41,6,0.43,1,N,236,236,2,3.5,1.0,0.5,0.00,0.00,0.3,5.30,383.0,383.0
1,1,1,2017-05-01 00:00:00,2017-05-01 00:25:18,1,17.10,2,N,132,141,1,52.0,0.0,0.5,10.55,0.00,0.3,63.35,530.0,383.0
2,2,1,2017-05-01 00:00:00,2017-05-01 00:22:42,1,9.90,1,N,138,50,1,29.0,0.5,0.5,7.20,5.76,0.3,43.26,283.0,383.0
3,3,2,2017-05-01 00:00:00,2017-05-01 00:27:42,1,4.40,1,N,48,7,2,19.5,0.5,0.5,0.00,0.00,0.3,20.80,383.0,280.0
4,4,2,2017-05-01 00:00:00,2017-05-01 00:00:00,1,1.81,1,N,186,233,2,18.0,0.0,0.5,0.00,0.00,0.3,18.80,383.0,383.0
5,5,2,2017-05-01 00:00:00,2017-05-01 15:38:37,2,0.00,5,N,264,142,1,60.0,0.0,0.0,0.00,0.00,0.0,60.00,0.0,383.0
6,6,1,2017-05-01 00:00:01,2017-05-01 00:16:31,1,3.60,1,N,211,161,2,15.0,0.5,0.5,0.00,0.00,0.3,16.30,356.0,304.0
7,7,1,2017-05-01 00:00:01,2017-05-01 00:09:58,1,1.80,1,N,48,229,2,9.0,0.5,0.5,0.00,0.00,0.3,10.30,383.0,305.0
8,8,1,2017-05-01 00:00:02,2017-05-01 00:18:49,1,4.10,1,N,142,144,2,16.0,0.5,0.5,0.00,0.00,0.3,17.30,383.0,357.0
9,9,2,2017-05-17 23:49:35,2017-05-17 23:54:34,6,0.85,1,N,170,163,2,5.5,1.0,0.5,0.00,0.00,0.3,7.30,286.0,383.0


In [ ]:
for i in range(len(taxi_trip_05)):
    current=taxi_trip_05['tpep_pickup_datetime'][i]
    current_time= datetime.datetime.strptime(current,"%Y-%m-%d %H:%M:%S")
    Day=cuurent_time.day
    Hour=cuurent_time.min

In [127]:
import collections
import math
taxi_halfhour_pick= np.zeros([31,48,729])#生成数组保存每组速度
taxi_halfhour_drop= np.zeros([31,48,729])#
for i in range(len(taxi_trip_05)):
    current=taxi_trip_05['tpep_pickup_datetime'][i]
    #print(current)
    current_time= datetime.datetime.strptime(current,"%Y-%m-%d %H:%M:%S")
    day=current_time.day
    hour=current_time.hour
    min=current_time.minute
    Time_min=hour*60+min
    interval=math.ceil(Time_min/30)-1
    taxi_halfhour_pick[day-1,interval,np.int(taxi_trip_05['PULocationGrid_ID'][i])]=taxi_halfhour_pick[day-1,interval,np.int(taxi_trip_05['PULocationGrid_ID'][i])]+1
    taxi_halfhour_drop[day-1,interval,np.int(taxi_trip_05['DOLocationGrid_ID'][i])]=taxi_halfhour_drop[day-1,interval,np.int(taxi_trip_05['DOLocationGrid_ID'][i])]+1

taxi_halfhour_pick= taxi_halfhour_pick.reshape([31,48,27,27])#生成数组保存每组速度
taxi_halfhour_drop= taxi_halfhour_drop.reshape([31,48,27,27])#

In [129]:
#映射到三维当中去
taxi_halfhour_pick1= np.zeros([1488,27,27])
taxi_halfhour_drop1= np.zeros([1488,27,27])
for i in range(0,31):
    for j in range(0,48):
        taxi_halfhour_pick1[i*48+j,:,:]=taxi_halfhour_pick[i,j,:,:]
        taxi_halfhour_drop1[i*48+j,:,:]=taxi_halfhour_drop[i,j,:,:]
for i in range(1488):
    taxi_halfhour_pick1[i,0,0]=0
    taxi_halfhour_drop1[i,0,0]=0

In [57]:
import pandas as pd
import numpy as np

In [62]:
taxi_data=pd.DataFrame(pd.read_csv('E:/data ming/accident_prediction/NYC_data/taxi_1705.csv'))

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
taxi_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Day,Unnamed: 4,pick_time,Unnamed: 6,Unnamed: 7,drop_time,Unnamed: 9,Unnamed: 10,Unnamed: 11,pick_ID,drop_ID
0,1.0,1,5.0,1.0,0.00000,0.00000,1.0,0.01757,25.30000,1,17.1,63.35,530,383
1,2.0,1,5.0,1.0,0.00000,0.00000,1.0,0.01576,22.70000,1,9.9,43.26,283,383
2,3.0,2,5.0,1.0,0.00000,0.00000,1.0,0.01924,27.70000,1,4.4,20.8,383,280
3,6.0,1,5.0,1.0,0.00001,0.01667,1.0,0.01147,16.51667,1,3.6,16.3,356,304
4,7.0,1,5.0,1.0,0.00001,0.01667,1.0,0.00692,9.96667,1,1.8,10.3,383,305
5,8.0,1,5.0,1.0,0.00002,0.03333,1.0,0.01307,18.81667,1,4.1,17.3,383,357
6,12.0,2,5.0,1.0,0.00002,0.03333,1.0,0.00810,11.66667,2,2.52,11.8,304,383
7,13.0,1,5.0,1.0,0.00003,0.05000,1.0,0.00304,4.38333,2,0.7,7.55,384,385
8,14.0,2,5.0,1.0,0.00003,0.05000,1.0,0.01182,17.01667,5,8.41,35.56,283,383
9,15.0,2,5.0,1.0,0.00003,0.05000,1.0,0.00829,11.93333,1,2.91,14.76,303,357


In [63]:
#对一个月的数据做处理 每天30min一个interval 一天48个 31*48
import collections
taxi_halfhour_pick= np.zeros([31,48,27,27])#生成数组保存每组速度
taxi_halfhour_drop= np.zeros([31,48,27,27])#
ss=np.int32(0)
for ii in range(1,32):
    taxi_dataset=taxi_data[taxi_data['Day']==np.float(ii)]
    count_half_hour=np.zeros(49)  #生成数组保存每组的个数，每半小时一个为48 下面为30分钟 27*27是网格大小
    for j in range(0,48):# 分段，计算每个interval中有几条记录 然后每次索引时用这个数据来控制归类的数据条目段
        ab=np.where((taxi_dataset['pick_time']>=30.0*j)&(taxi_dataset['pick_time']<30*j+30),1,0)
        count_half_hour[j+1]=collections.Counter(ab)[1]
    for j in range(0,48):
        #speed_value_ingrid=np.zeros(729)
        count_pick_id=np.zeros(729)
        count_drop_id=np.zeros(729)
        for i in range(ss,ss+np.int32(count_half_hour[j+1])):
           #print(ii,i,j)
            count_pick_id[np.int(taxi_dataset['pick_ID'][i])]=count_pick_id[np.int(taxi_dataset['pick_ID'][i])]+1
            count_drop_id[np.int(taxi_dataset['drop_ID'][i])]=count_pick_id[np.int(taxi_dataset['drop_ID'][i])]+1
        ss=np.int32(ss)+np.int32(count_half_hour[j+1])#ss为总的累计值
        #print(ss)
       
        taxi_halfhour_pick[ii-1,j,:,:]=count_pick_id.reshape(27,27)
        taxi_halfhour_drop[ii-1,j,:,:]=count_drop_id.reshape(27,27)
    print(ss)

#映射到三维当中去
taxi_halfhour_pick1= np.zeros([1488,27,27])
taxi_halfhour_drop1= np.zeros([1488,27,27])
for i in range(0,31):
    for j in range(0,48):
        taxi_halfhour_pick1[i*48+j,:,:]=taxi_halfhour_pick[i,j,:,:]
        taxi_halfhour_drop1[i*48+j,:,:]=taxi_halfhour_drop[i,j,:,:]

303358
635923
973099
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003371
1003373
1003375
1003375
1003375
1048573
1048573
1048573
1048573
1048573
1048573
1048573
1048573
1048573
1048573
1048573


In [ ]:
for i in range(1488):
    taxi_halfhour_pick1[i,0,0]=0
    taxi_halfhour_drop1[i,0,0]=0


In [130]:
# taxi_halfhour_pick1= np.zeros([1488,27,27])
# taxi_halfhour_drop1= np.zeros([1488,27,27])
# for i in range(0,31):
#     for j in range(0,48):
#         taxi_halfhour_pick1[i*48+j,:,:]=taxi_halfhour_pick[i,j,:,:]
#         taxi_halfhour_drop1[i*48+j,:,:]=taxi_halfhour_drop[i,j,:,:]
        
taxi_halfhour_pick1_05=np.zeros([354,1488])
taxi_halfhour_drop1_05=np.zeros([354,1488])
for i in range(1488):
    taxi_halfhour_pick1_05[:,i]=transfer_354(taxi_halfhour_pick1[i,:,:])
    taxi_halfhour_drop1_05[:,i]=transfer_354(taxi_halfhour_drop1[i,:,:])
np.save('taxi_halfhour_pick_05.npy',taxi_halfhour_pick1_05)
np.save('taxi_halfhour_drop_05.npy',taxi_halfhour_drop1_05)

In [ ]:
#taxi_trips 划分到网格 制作矩阵

In [147]:
Interval=1488 
DAY=31
#Interval=1440
#DAY=30
#Interval=1344
#DAY=28

In [133]:
taxi_trip_04

,Dispatching_base_num,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID
0,B00001,2017-01-01 00:30:00,NaN,NaN,NaN
1,B00001,2017-01-01 00:30:00,NaN,NaN,NaN
2,B00008,2017-01-01 00:50:00,NaN,NaN,NaN
3,B00009,2017-01-01 00:10:00,NaN,NaN,NaN
4,B00009,2017-01-01 00:50:00,NaN,NaN,NaN
5,B00014,2017-01-01 00:10:00,NaN,NaN,NaN
6,B00014,2017-01-01 00:05:00,NaN,NaN,NaN
7,B00014,2017-01-01 00:10:00,NaN,NaN,NaN
8,B00014,2017-01-01 00:50:00,NaN,NaN,NaN
9,B00014,2017-01-01 00:15:00,NaN,NaN,NaN


In [142]:
import numpy as np
import pandas as pd
taxi_trip_04=pd.read_csv(r'E:\data ming\accident_prediction\NYC_data\taxi_data\green_tripdata_2017-01.csv') #改名字！

path="E:/data ming/accident_prediction/NYC_data/taxi_data/taxi_loca11.csv"
all_data=pd.DataFrame(pd.read_csv(path))
index_num=np.zeros([266,2]) #建立空矩阵 存放索引号-索引内容
index_num[all_data['LocationID'],0]=all_data['LocationID'] #填充索引内容
index_num[all_data['LocationID'],1]=all_data['grid_id'] #填充索引内容

#path="E:/data ming/accident_prediction/NYC_data/taxi_data/yellow_tripdata_2017-01.csv"
taxi_data=taxi_trip_04
#索引成功后的数组
PULocationGrid_ID=[]
DOLocationGrid_ID=[]
for i in range(0,len(taxi_data)):
  #  try:
        PULocationGrid_ID.append(index_num[np.int(taxi_data['PULocationID'][i]),1])
        DOLocationGrid_ID.append(index_num[taxi_data['DOLocationID'][i],1])
 #   except:
        #PULocationGrid_ID.append(0)
        #DOLocationGrid_ID.append(index_num[taxi_data['DOLocationID'][i],1])
#保存为原矩阵中的内容，增加两列。
taxi_data['PULocationGrid_ID']=PULocationGrid_ID  
#taxi_data['DOLocationGrid_ID']=DOLocationGrid_ID  
#将结果输出
Path='E:/data ming/accident_prediction/NYC_data/green_taxi_1701.csv'
pd.DataFrame(taxi_data).to_csv(Path)#直接输出网格index #改名字！

In [144]:
taxi_data['DOLocationGrid_ID']=DOLocationGrid_ID 

In [145]:
taxi_data

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,PULocationGrid_ID,DOLocationGrid_ID
0,2,2017/1/1 0:01,2017/1/1 0:11,N,1,42,166,383.0,197.0
1,2,2017/1/1 0:03,2017/1/1 0:09,N,1,75,74,383.0,383.0
2,2,2017/1/1 0:04,2017/1/1 0:12,N,1,82,70,336.0,283.0
3,2,2017/1/1 0:01,2017/1/1 0:14,N,1,255,232,383.0,384.0
4,2,2017/1/1 0:00,2017/1/1 0:18,N,1,166,239,197.0,383.0
5,2,2017/1/1 0:00,2017/1/1 0:13,N,1,179,226,280.0,334.0
6,2,2017/1/1 0:02,2017/1/1 0:26,N,1,74,167,383.0,173.0
7,2,2017/1/1 0:15,2017/1/1 0:28,N,1,112,37,360.0,383.0
8,2,2017/1/1 0:06,2017/1/1 0:11,N,1,36,37,383.0,383.0
9,2,2017/1/1 0:14,2017/1/1 0:28,N,1,127,174,91.0,66.0


In [150]:
import collections
import math
taxi_halfhour_pick= np.zeros([DAY,48,729])#生成数组保存每组速度
taxi_halfhour_drop= np.zeros([DAY,48,729])#
for i in range(len(taxi_data)):
    current_pick=taxi_data['lpep_pickup_datetime'][i]  
    #print(current)
    current_time= datetime.datetime.strptime(current_pick,"%Y/%m/%d %H:%M")
    day=current_time.day
    hour=current_time.hour
    min=current_time.minute
    Time_min=hour*60+min
    interval_pick=math.ceil(Time_min/30)-1
    
    current_drop=taxi_data['lpep_dropoff_datetime'][i]  
    #print(current)
    current_time= datetime.datetime.strptime(current_drop,"%Y/%m/%d %H:%M")
    day=current_time.day
    hour=current_time.hour
    min=current_time.minute
    Time_min=hour*60+min
    interval_drop=math.ceil(Time_min/30)-1
    taxi_halfhour_pick[day-1,interval_pick,np.int(taxi_data['PULocationGrid_ID'][i])]=taxi_halfhour_pick[day-1,interval_pick,np.int(taxi_data['PULocationGrid_ID'][i])]+1
    taxi_halfhour_drop[day-1,interval_drop,np.int(taxi_data['DOLocationGrid_ID'][i])]=taxi_halfhour_drop[day-1,interval_drop,np.int(taxi_data['DOLocationGrid_ID'][i])]+1

taxi_halfhour_pick= taxi_halfhour_pick.reshape([DAY,48,27,27])#生成数组保存每组速度
taxi_halfhour_drop= taxi_halfhour_drop.reshape([DAY,48,27,27])#

In [ ]:
#映射到三维当中去
taxi_halfhour_pick1= np.zeros([Interval,27,27])
taxi_halfhour_drop1= np.zeros([Interval,27,27])
for i in range(0,DAY):
    for j in range(0,48):
        taxi_halfhour_pick1[i*48+j,:,:]=taxi_halfhour_pick[i,j,:,:]
        taxi_halfhour_drop1[i*48+j,:,:]=taxi_halfhour_drop[i,j,:,:]
for i in range(Interval):
    taxi_halfhour_pick1[i,0,0]=0
    taxi_halfhour_drop1[i,0,0]=0

In [ ]:
index_road=pd.DataFrame(pd.read_csv("E:/data ming/accident_prediction/NYC_data/Experimental_data/index_road.csv"))
index_road=np.array(index_road)[:,0]

def transfer_354(A):
    B1=np.zeros([354])
    A=np.array(A).reshape([729])
    for i in range(354):
        B1[i]=A[index_road[i]]
    return B1


taxi_halfhour_pick1_04=np.zeros([354,Interval])
taxi_halfhour_drop1_04=np.zeros([354,Interval])
for i in range(Interval):
    taxi_halfhour_pick1_04[:,i]=transfer_354(taxi_halfhour_pick1[i,:,:])
    taxi_halfhour_drop1_04[:,i]=transfer_354(taxi_halfhour_drop1[i,:,:])
np.save('taxi_pick_01.npy',taxi_halfhour_pick1_04)
np.save('taxi_drop_01.npy',taxi_halfhour_drop1_04)